루브릭
1. GAN의 두 모델 구조를 통해 이미지를 성공적으로 생성하였다.  
오브젝트 종류를 육안으로 구별할 수 있을 만한 이미지를 생성하였다.
  
2. 생성 이미지 시각화 및 학습 그래프를 통해 GAN학습이 바르게 진행되었음을 입증하였다.  
gif를 통해 생성이미지 품질이 서서히 향상되는 것과, fake accuracy가 추세적으로 0.5를 향해 하향하고 있음을 확인하였다.
  
3. 추가적인 GAN모델구조 혹은 학습과정 개선 아이디어를 제안하고 이를 적용하였다.  
모델구조 변경 혹은 학습과정 개선 아이디어를 추가적으로 적용해보았고, 적용 전과 비교하였을 때의 내용을 표와 같은 시각적 자료로 작성하였다.

노드 예제와 다른 점: 데이터의 shape, 색상 채널(grayscale->color), 입력데이터 차원이 증가(채널이 증가하므로)  

절차  
작업환경 및 데이터셋 구성  
-import, dataset 불러오기, 정규화, 학습 데이터 확인, tf.data.Dataset 모듈의 from_tensor_slices() 함수를 사용하여 미니배치 데이터셋을 구성  
생성자 모델 구현  
-생성자 모델 구현 함수 작성, noise = tf.random.normal([1, 100])로 생성된 랜덤 노이즈를 입력으로 하여 방금 구현한 생성자로 랜덤 이미지를 생성  
판별자 모델 구현  
손실함수 및 최적화 함수 구현  
-optimizer도 구현  
훈련과정 상세 기능 구현  
-훈련과정 처리, 훈련과정 동안 생성한 이미지 시각화, 이미지 판별 history 시각화, checkpoint모듈 설정  
학습 과정 진행  
-50 epoch이상, 시각화 표현 , 학습과정 담은 샘플 이미지, gif파일, 학습 진행 그래프 이미지 함께 제출
훈련 과정 개선  

In [1]:
import os
import glob
import time

import PIL
import imageio
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline

print("tensorflow", tf.__version__)

tensorflow 2.9.1
